<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 
<hr><hr><hr>

## Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import environment as E
from buffers import ReplayBuffer, nStepBuffer
from agent import D4PG_Agent

#from get_args import get_args

import os.path
import time
import re
import sys
import importlib
import copy

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
import multiprocessing as multi
multi.cpu_count()

16

## Manually declare an ARGS class
<i> For testing code in the notebook, to take the place of argparser in the command line.

In [2]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        
args = Args()

<b>Confirm that the args are all set the way we want them.

In [3]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000


## Load the environment
<i> & print a bit of information contained in the wrapper class

In [4]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


<hr>

# Test code as it's developed

<hr>

## Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [127]:
env.train = False
env.reset()

In [132]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
actions = np.zeros((20,4))
for i in range(100):
    #actions = np.random.randn(env.agent_count, env.action_size)
    actions[:,3] += .01
    actions = np.clip(actions, -1, 1)
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.020499999541789292


In [125]:
actions = np.zeros((20,4))
for i in range(10):
    actions[:,0] += .1
    print(actions)

[[0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]
 [0.1 0.  0.  0. ]]
[[0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]
 [0.2 0.  0.  0. ]]
[[0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  0.  0. ]
 [0.3 0.  

### Force-reload modules as they're updated
<i> This notebook was developed as the code is written in Atom, the below cell reloads the modules as they're needed.

In [ ]:
import agent
importlib.reload(agent)
#importlib.reload(E)
importlib.reload(models)
from agent import D4PG_Agent

In [11]:
d4pg_agent = D4PG_Agent(env.state_size, env.action_size, env.agent_count)
# print(d4pg_agent.__class__.__name__)
# print(d4pg_agent.memory)
# agent.initialize_memory(10, env)

In [12]:
env.reset()
states = env.states

### Test out Actor actions without training
<i> Test the <b>Actor</b> network

In [112]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(30):
    actions = d4pg_agent.act(states)
    print("ACTIONS:", actions[1])
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

ACTIONS: [ 0.12072824 -0.08650129 -0.2426841   0.20263317]
ACTIONS: [ 0.11929604  0.0774109   0.15026315 -0.14079183]
ACTIONS: [-0.21552052  0.3036239  -0.05686849  0.16493551]
ACTIONS: [-0.37239876  0.64675504 -0.02954452 -0.41759738]
ACTIONS: [-0.40439492  0.49016446 -0.09638876  0.5759712 ]
ACTIONS: [ 0.18600334 -0.13982823 -0.18315789 -0.1067877 ]
ACTIONS: [ 0.10396563  0.20145947 -0.04876871  0.49385545]
ACTIONS: [-0.18973796 -0.23842593 -0.01363463  0.26116577]
ACTIONS: [ 0.12457623 -0.04857243  0.06835352 -0.2388278 ]
ACTIONS: [-0.08805268 -0.5120842  -0.18368335 -0.21154818]
ACTIONS: [ 0.22746587  0.29810438 -0.20771603 -0.5380932 ]
ACTIONS: [ 0.32480836  0.5606363   0.04752121 -0.37489942]
ACTIONS: [ 0.25482413 -0.12632596  0.11426903 -0.23253416]
ACTIONS: [-0.04591284 -0.48473155 -0.12828822 -0.40005875]
ACTIONS: [-0.12775998  0.27066866 -0.02793056  0.14100502]
ACTIONS: [ 0.0389617  -0.6315945   0.14557207 -0.23061852]
ACTIONS: [-0.28976026 -0.40605357  0.5128111   0.3156206

ACTIONS: [-0.19308537 -0.14845064 -0.40432417 -0.4416672 ]
ACTIONS: [ 0.1012351  -0.31623068  0.00547595 -0.19404681]
ACTIONS: [-0.2970166  -0.18865255 -0.12186845 -0.30413297]
ACTIONS: [ 0.40719062 -0.36198708 -0.18326634 -0.30867195]
ACTIONS: [-0.23754944 -0.3127232  -0.5600174  -0.10123626]
ACTIONS: [-0.3259422  -0.25763902  0.34360456 -0.07229388]
ACTIONS: [-0.2848015   0.2072718   0.21084322  0.13098921]
ACTIONS: [ 0.5999527  -0.2404937  -0.6527139   0.37243307]
ACTIONS: [ 0.29044706  0.03963693 -0.39005908  0.14409314]
ACTIONS: [-0.61647457  0.00718306 -0.09980967  0.58261085]
ACTIONS: [-0.20516203 -0.07536345  0.01494337  0.08495621]
ACTIONS: [-0.01981568  0.17704473 -0.22985974 -0.44495985]
ACTIONS: [-0.13787696 -0.21050553 -0.10659299 -0.031178  ]
ACTIONS: [ 0.16463126 -0.23933665  0.11565618  0.28922215]
ACTIONS: [ 0.07802901 -0.52407855 -0.13504197 -0.5090258 ]
ACTIONS: [-0.29265043 -0.42298955 -0.3449492   0.54416215]
ACTIONS: [-0.20898987 -0.14429957  0.24299993 -0.6019062

ACTIONS: [ 0.01999115 -0.07211079  0.05800765 -0.7156198 ]
ACTIONS: [-0.1131888  -0.03167914  0.39704704  0.04985125]
ACTIONS: [ 0.1707986   0.17535986 -0.18388934  0.18508905]
ACTIONS: [ 0.34602582 -0.5155452  -0.3216077  -0.49399054]
ACTIONS: [ 0.21240127  0.220057    0.0998925  -0.05200554]
ACTIONS: [-0.42082015  0.5404009   0.3127645   0.32569203]
ACTIONS: [-0.01972523  0.5407506   0.16946614 -0.04559203]
ACTIONS: [ 0.20376284  0.05253517  0.38212094 -0.17802615]
ACTIONS: [-0.49493235  0.14781837  0.41671538 -0.36746293]
ACTIONS: [ 0.22824979  0.27843413 -0.5000596  -0.571131  ]
ACTIONS: [ 0.40300834  0.18392001  0.27293563 -0.38836813]
ACTIONS: [ 0.25405303 -0.16768111 -0.22785416 -0.27156442]
ACTIONS: [-0.12524925  0.40104526  0.41193968  0.19621332]
ACTIONS: [-0.13101798 -0.2090228   0.11435518 -0.13032082]
ACTIONS: [ 0.18751849 -0.17107828  0.28036445  0.40508705]
ACTIONS: [-0.38494578 -0.21333385 -0.07074345  0.02364682]
ACTIONS: [-0.32675585 -0.18471791 -0.04540517  0.3133314

### Test out Critic scores without training
<i> Test the <b>Critic</b> network

In [133]:
env.reset()
scores = np.zeros(env.agent_count)
states = env.states
for i in range(10):
    actions = d4pg_agent.act(states) 
    next_states, rewards, dones = env.step(actions)
    scores += rewards
    q, probs = d4pg_agent.critic(next_states, torch.from_numpy(actions))
    print(probs[0])
    print(q[0])
    #print(values.sample())
    states = next_states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

tensor([0.0196, 0.0197, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0195, 0.0196,
        0.0196, 0.0197, 0.0196, 0.0196, 0.0197, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0197, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0197, 0.0196,
        0.0196, 0.0197, 0.0197, 0.0196, 0.0195, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0196, 0.0196, 0.0197, 0.0196, 0.0196, 0.0196],
       grad_fn=<SelectBackward>)
tensor(-0.0014, grad_fn=<SelectBackward>)
tensor([0.0196, 0.0197, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0196, 0.0197, 0.0196, 0.0196, 0.0197, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0197, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0196, 0.0197, 0.0196,
        0.0196, 0.0197, 0.0197, 0.0196, 0.0195, 0.0196, 0.0196, 0.0196, 0.0196,
        0.0196, 0.0196, 0.0197, 0.0196, 0.0196, 0.0196],
       grad

## Learn about how the Categorical Bellman step works

In [ ]:
actions = np.random.randn(env.agent_count, env.action_size)
actions = np.clip(actions, -1, 1).astype(np.float32)
states = torch.from_numpy(env.states).float()

In [ ]:
q, probs, log_probs = critic(states, torch.from_numpy(actions))

In [ ]:
print(probs.shape)
print(log_probs.shape)
print(q)

In [49]:
rewards

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [136]:
class Container():
    def __init__(self):
        pass
c = Container()
    

In [151]:
c.vmin = -10
c.vmax = 10
c.natoms = 51
c.gamma = .99
c.atoms = torch.linspace(vmin, vmax, natoms)
c.delta_z = (vmax - vmin) / (natoms -1)
c.r = torch.tensor(rewards).unsqueeze(-1)

c.probs = probs.detach()
c.q_next = (probs * atoms).sum()


In [142]:
def print_vars(x):
    for arg in vars(x):
        print("{}: {}".format(arg.upper(), getattr(x, arg)))

In [218]:
c.atoms

tensor([-10.0000,  -9.6000,  -9.2000,  -8.8000,  -8.4000,  -8.0000,  -7.6000,
         -7.2000,  -6.8000,  -6.4000,  -6.0000,  -5.6000,  -5.2000,  -4.8000,
         -4.4000,  -4.0000,  -3.6000,  -3.2000,  -2.8000,  -2.4000,  -2.0000,
         -1.6000,  -1.2000,  -0.8000,  -0.4000,   0.0000,   0.4000,   0.8000,
          1.2000,   1.6000,   2.0000,   2.4000,   2.8000,   3.2000,   3.6000,
          4.0000,   4.4000,   4.8000,   5.2000,   5.6000,   6.0000,   6.4000,
          6.8000,   7.2000,   7.6000,   8.0000,   8.4000,   8.8000,   9.2000,
          9.6000,  10.0000])

### projected atoms
<html><i>
<b>atoms.view(1,-1)</b> becomes shape [1, num_atoms]
<br>
<b>r</b> is unsqueezed in the last (-1) dimension, so it's shape [20,1]
<br>
the result is a tensor that holds an offset (projected) version of the atoms for each reward instance

In [183]:
np.set_printoptions(suppress=True)

#### tz = projected atoms, atoms (values) projected by scaling and offsetting via the bellman equation


In [149]:
tz = c.r + c.gamma * c.atoms.view(1,-1)
tz.clamp_(vmin, vmax)

tensor([[-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        ...,
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000],
        [-9.9000, -9.5040, -9.1080,  ...,  9.1080,  9.5040,  9.9000]])

#### computes "bj" from the pseudo-code

In [215]:
b = (tz - c.vmin) / c.delta_z
b[0].numpy()

array([ 0.25000095,  1.2399983 ,  2.2300005 ,  3.2200003 ,  4.21      ,
        5.2       ,  6.1899996 ,  7.1800003 ,  8.169999  ,  9.160001  ,
       10.15      , 11.139999  , 12.13      , 13.12      , 14.11      ,
       15.099999  , 16.09      , 17.08      , 18.07      , 19.06      ,
       20.050001  , 21.04      , 22.03      , 23.02      , 24.01      ,
       25.        , 25.990002  , 26.98      , 27.97      , 28.960001  ,
       29.949999  , 30.94      , 31.93      , 32.92      , 33.91      ,
       34.899998  , 35.890003  , 36.88      , 37.87      , 38.86      ,
       39.850002  , 40.839996  , 41.830006  , 42.82      , 43.81      ,
       44.8       , 45.79      , 46.780003  , 47.770004  , 48.760002  ,
       49.75      ], dtype=float32)

#### l/u in the psuedocode are LOWER and UPPER bounds on the supports

In [205]:
l = b.floor().long()
u = b.ceil().long()
print(l[0])
print(u[0])

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])


#### m_l/m_u are computed in the pseudocode under "distribute the probability of tz", but still a bit opaque to me on how it should be used

In [216]:
# version from GITHUB implementations

gdml = (u.float() + (l == u).float() - b) * c.probs
gdml[0].numpy()

array([0.01467833, 0.01493664, 0.01510203, 0.0153105 , 0.01546834,
       0.01571159, 0.01590588, 0.0160307 , 0.01628518, 0.01649079,
       0.01670613, 0.01687601, 0.01705097, 0.01730292, 0.01742529,
       0.01766971, 0.01783578, 0.0180223 , 0.01823092, 0.01840046,
       0.0186618 , 0.01879957, 0.01903829, 0.01920612, 0.0193745 ,
       0.01964528, 0.00019637, 0.00039313, 0.00058676, 0.00078374,
       0.00098073, 0.00117412, 0.00136963, 0.00156546, 0.00176948,
       0.00196226, 0.00215623, 0.00235884, 0.00255588, 0.0027378 ,
       0.00293237, 0.00312979, 0.00333509, 0.00352029, 0.00372426,
       0.00392292, 0.00411897, 0.00432351, 0.00451106, 0.00469343,
       0.0049065 ], dtype=float32)

In [206]:
# version from GITHUB implementations

gdmu = (b - l.float()) * c.probs
gdmu[0].numpy()

array([0.0048928 , 0.00471679, 0.00451101, 0.00431835, 0.00411184,
       0.00392789, 0.003731  , 0.00351894, 0.0033355 , 0.00314112,
       0.00294813, 0.00274724, 0.00254785, 0.00235949, 0.00215368,
       0.00196329, 0.00176398, 0.00156716, 0.00137221, 0.00117449,
       0.00098222, 0.00078333, 0.00058883, 0.00039197, 0.00019571,
       0.        , 0.01944361, 0.01926299, 0.01897159, 0.01881013,
       0.01863343, 0.01839465, 0.01819655, 0.01800233, 0.01789142,
       0.01765985, 0.01744645, 0.01729832, 0.0171046 , 0.01681799,
       0.01661707, 0.01643098, 0.01628374, 0.01603686, 0.01587726,
       0.01569162, 0.01549527, 0.01532905, 0.01510262, 0.01486269,
       0.01471951], dtype=float32)

In [207]:
# version from the PAPER

pdml = l.float() + (c.probs * (u.float()-b))
pdml[0].numpy()

array([ 0.01467833,  1.0149367 ,  2.0151021 ,  3.0153105 ,  4.015468  ,
        5.015712  ,  6.015906  ,  7.016031  ,  8.016285  ,  9.016491  ,
       10.016706  , 11.016876  , 12.017051  , 13.0173025 , 14.017426  ,
       15.01767   , 16.017836  , 17.018023  , 18.01823   , 19.0184    ,
       20.018661  , 21.018799  , 22.01904   , 23.019207  , 24.019375  ,
       25.        , 25.000196  , 26.000393  , 27.000587  , 28.000784  ,
       29.00098   , 30.001175  , 31.00137   , 32.001564  , 33.00177   ,
       34.00196   , 35.002155  , 36.002357  , 37.002556  , 38.00274   ,
       39.002934  , 40.003128  , 41.003334  , 42.00352   , 43.003723  ,
       44.00392   , 45.00412   , 46.004322  , 47.004513  , 48.004692  ,
       49.004906  ], dtype=float32)

In [208]:
# version from the PAPER

pdmu = u.float() + (c.probs * (b - l.float()))
pdmu[0].numpy()

array([ 1.0048928,  2.0047169,  3.004511 ,  4.004318 ,  5.004112 ,
        6.0039277,  7.003731 ,  8.003519 ,  9.003336 , 10.003141 ,
       11.002948 , 12.002748 , 13.002548 , 14.002359 , 15.002153 ,
       16.001963 , 17.001764 , 18.001568 , 19.001371 , 20.001175 ,
       21.000982 , 22.000784 , 23.00059  , 24.000393 , 25.000196 ,
       25.       , 26.019444 , 27.019262 , 28.018972 , 29.01881  ,
       30.018633 , 31.018394 , 32.018196 , 33.018    , 34.01789  ,
       35.01766  , 36.017445 , 37.0173   , 38.017105 , 39.01682  ,
       40.016617 , 41.01643  , 42.016285 , 43.016037 , 44.015877 ,
       45.01569  , 46.015495 , 47.015327 , 48.015102 , 49.014862 ,
       50.01472  ], dtype=float32)

In [202]:
target_prob.size(0)

20

In [209]:
target_prob = torch.tensor(np.zeros(probs.size()))
for i in range(target_prob.size(0)):
    target_prob[i].index_add_(0, l[i].long(), gdml[i].double())
    target_prob[i].index_add_(0, u[i].long(), gdmu[i].double())
target_prob[0].numpy()

array([0.01467833, 0.01982944, 0.01981882, 0.01982151, 0.01978669,
       0.01982343, 0.01983377, 0.0197617 , 0.01980412, 0.01982629,
       0.01984725, 0.01982415, 0.01979821, 0.01985077, 0.01978477,
       0.01982339, 0.01979907, 0.01978629, 0.01979808, 0.01977267,
       0.01983629, 0.0197818 , 0.01982162, 0.01979494, 0.01976647,
       0.02003736, 0.01983674, 0.01984975, 0.01975532, 0.01979086,
       0.01980755, 0.01976428, 0.01976201, 0.01977182, 0.01985367,
       0.01981608, 0.01980529, 0.01985421, 0.0198424 , 0.01975036,
       0.01974687, 0.01976607, 0.01980403, 0.01976112, 0.01980018,
       0.01981059, 0.01981879, 0.01984012, 0.01979605, 0.0197692 ,
       0.01471951])

In [ ]:
# zip dml and dmu together offset by 1 index and subtract 1, receive amount to add to probs!
x = zip()

In [200]:
c.probs[0].numpy()

array([0.01957113, 0.01965343, 0.01961304, 0.01962886, 0.01958018,
       0.01963948, 0.01963687, 0.01954965, 0.01962068, 0.01963191,
       0.01965426, 0.01962326, 0.01959882, 0.0196624 , 0.01957897,
       0.019633  , 0.01959976, 0.01958946, 0.01960313, 0.01957494,
       0.01964402, 0.01958291, 0.01962711, 0.01959809, 0.01957021,
       0.01964528, 0.01963998, 0.01965612, 0.01955835, 0.01959387,
       0.01961417, 0.01956877, 0.01956618, 0.01956779, 0.0196609 ,
       0.0196221 , 0.01960268, 0.01965716, 0.01966049, 0.01955579,
       0.01954945, 0.01956077, 0.01961883, 0.01955715, 0.01960152,
       0.01961454, 0.01961425, 0.01965257, 0.01961369, 0.01955612,
       0.01962601], dtype=float32)

In [ ]:
env.close()